In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [6]:
def getSubarray(dims, new_dims, offset=[0, 0, 0], spacing=1):

    idx_column = np.arange(0, dims[0], spacing).reshape(-1,1)
    idx_row = np.arange(0, dims[1], spacing)
    idx_freq = np.arange(offset[2], offset[2] + new_dims[2], 1)

    if (len(idx_column) < new_dims[0]) or (len(idx_row) < new_dims[1]):
        print('Problem in finding the subarray')
        exit()
    else:
        idx_column = idx_column[offset[0]:offset[0] + new_dims[0]]

    idx_array = np.zeros((new_dims[0] * new_dims[1], 1), dtype=int)

    for il2 in range(new_dims[1]):
        idx_array[il2 * new_dims[0]:(il2 + 1) * new_dims[0]] = idx_column + dims[0] * (il2 + offset[1]) * spacing

    return idx_array.reshape(-1), idx_freq
    
def delay_response_vector_naive(az, tau, r, f, wave):
    a = np.zeros(r.shape[1], dtype=complex)
    e = np.matrix([np.cos(az), np.sin(az)])

    for i in range(len(a)):
        a[i] = np.exp(2j * np.pi * wave * np.dot(e, r[:, i]))
    
    b = np.zeros(len(f), dtype=complex)
    for i in range(len(b)):
        b[i] = np.exp(-2j * np.pi * f[i] * tau)

    return np.kron(b, a)

def delay_response_vector(az, tau, r, f, wave):
    # Angle
    e = np.matrix([np.cos(az), np.sin(az)])
    a = np.exp(2j * np.pi * (1 / wave) * e @ r).T
    
    # Time delay
    b = np.exp(-2j * np.pi * f * tau)
    
    return np.kron(b, a)

def get_noise(x, SNR_dB=5):
    L, N = x.shape
    SNR = 10.0**(SNR_dB / 10.0) # Desired linear SNR
    xVar = x.var() # Power of signal
    nVar = xVar / SNR # Desired power of noise
    n = np.random.normal(0, np.sqrt(nVar*2.0)/2.0, size=(L, 2*N)).view(complex)
    
    #print('varX = ', xVar, 'varY = ', nVar)
    #print(10*np.log10(xVar / nVar), SNR_dB)
    
    return n

def spatialSmoothing(X, L, Ls, method=None):
    P1, P2, P3 = L - Ls + 1
    RF = np.zeros((np.prod(Ls), np.prod(Ls)), dtype=complex)

    for p1 in range(P1):
        for p2 in range(P2):
            for p3 in range(P3):
                
                idx_array, idx_n = getSubarray(L, Ls, offset=[p1, p2, p3])
                  
                x_sm = X[idx_array][:, idx_n].flatten(order='F')

                x_sm = x_sm.reshape(-1, 1)

                RF += x_sm @ x_sm.conj().T

    RF = RF / (P1 * P2 * P3)

    if method is None:
        return RF
    else:
        J = np.flipud(np.eye(np.prod(Ls)))
        RFB = 1/2 * (RF + J @ RF.conj().T @ J)
        return RFB

# System parameters
SNR_dB = 50
N_rows = 71
N_cols = 66
array_size = np.array([N_rows, N_cols, 101])

# Subarray
L1 = 20
L2 = 20
L3 = 50
sub_array_size = np.array([L1, L2, L3])
idx_array, idx_n = getSubarray(array_size, sub_array_size, spacing=2)

# Smoothing subarray
Ls1 = 10
Ls2 = 10
Ls3 = 25
sub_array_smooth_size = np.array([Ls1, Ls2, Ls3])

# Load data
data = loadmat('MeasurementforMiniproject.mat')
X = data['X_synthetic'][idx_array][:, idx_n]
X += get_noise(X, SNR_dB)
r = data['r'][:, idx_array]
f = data['f'][idx_n]
f0 = data['f0'][0, 0]
tau = data['tau']
true_angles = np.rad2deg(data['smc_param'][0][0][1])
true_angles[true_angles < 0] = true_angles[true_angles < 0] + 360
RFB = spatialSmoothing(X, sub_array_size, sub_array_smooth_size)

# Print info
print(f'X: {X.shape}, r: {r.shape}')
print('f:', f.shape)
print('r:', r.shape)
print('Maximum time-delay:', tau[-1] - tau[0])
print('Number of sources:', data['smc_param'][0][0][0])
print('True angles:\n', true_angles)
print('True time-delays:\n', data['smc_param'][0][0][2])
print('RFB:', RFB.shape)

X: (400, 50), r: (2, 400)
f: (50, 1)
r: (2, 400)
Maximum time-delay: [5.e-07]
Number of sources: [[5]]
True angles:
 [[ 50.88252107]
 [ 34.82704776]
 [164.39983394]
 [272.69385129]
 [ 64.56419748]]
True time-delays:
 [[3.23578932]
 [4.39589382]
 [9.33539943]
 [9.35254152]
 [5.7079572 ]]
RFB: (2500, 2500)


In [10]:
# Entire X
def estimate_M(x):
    L, N = x.shape
    R = x @ x.conj().T / N
    E, _ = np.linalg.eig(R)
    idx = np.argsort(E)
    E = np.flipud(E[idx])
    pn = L - 1

    p = np.arange(1, pn+1).reshape([1, pn])
    M_est = N * np.log(E[np.arange(pn)]) + 1/2 * (p**2 + p) * np.log(N)
    
    return np.argmin(M_est)

print(estimate_M(data['X_synthetic']))

15


In [11]:
# Subarray
def estimate_M(x):
    L, N = x.shape
    R = x @ x.conj().T / N
    E, _ = np.linalg.eig(R)
    idx = np.argsort(E)
    E = np.flipud(E[idx])
    pn = L - 1

    p = np.arange(1, pn+1).reshape([1, pn])
    M_est = N * np.log(E[np.arange(pn)]) + 1/2 * (p**2 + p) * np.log(N)
    
    return np.argmin(M_est)

print(estimate_M(X))

6


In [12]:
# Spatial smoothed
def estimate_M(R, L, N):
    E, _ = np.linalg.eig(R)
    idx = np.argsort(E)
    E = np.flipud(E[idx])
    pn = L - 1

    p = np.arange(1, pn+1).reshape([1, pn])
    M_est = N * np.log(E[np.arange(pn)]) + 1/2 * (p**2 + p) * np.log(N)
    
    return np.argmin(M_est)

L, N = X.shape
print(estimate_M(RFB, L, N))

7
